In [ ]:
# time elapsed: 2047s

In [8]:
import random as rnd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import make_blobs
from math import sqrt
import seaborn as sns
import CLUEstering as clue
import uproot
import awkward as ak
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, MaxAbsScaler, QuantileTransformer
import os
from scipy.stats import zscore
from itertools import product
import plotly.express as px
from collections import Counter
import csv


def scale_x2_by_event_min_max(event_data, event_stats):
    # Use pre-calculated min/max values from event_stats for scaling
    min_x2 = event_stats['x2']['min']
    max_x2 = event_stats['x2']['max']
    
    event_data['x2'] = (event_data['x2'] - min_x2) / (max_x2 - min_x2)
    return event_data

def inverse_scale_x2_by_event_min_max(event_data, event_stats):
    # Use pre-calculated min/max values from event_stats for inverse scaling
    min_x2 = event_stats['x2']['min']
    max_x2 = event_stats['x2']['max']
    
    event_data['x2'] = event_data['x2'] * (max_x2 - min_x2) + min_x2
    return event_data

def normalize_x2_by_event_z_score(event_data, event_stats):
    # Use pre-calculated mean and std from event_stats for z-score normalization
    mean_x2 = event_stats['x2']['mean']
    std_x2 = event_stats['x2']['std']
    
    # Perform Z-score normalization using pre-calculated mean and std
    event_data['x2'] = (event_data['x2'] - mean_x2) / std_x2
    return event_data

def inverse_normalize_x2_by_event_z_score(event_data, event_stats):
    # Use pre-calculated mean and std from event_stats for the inverse z-score
    mean_x2 = event_stats['x2']['mean']
    std_x2 = event_stats['x2']['std']
    
    # Reverse the Z-score normalization using pre-calculated mean and std
    event_data['x2'] = event_data['x2'] * std_x2 + mean_x2
    return event_data
    
def scale_all_features_by_event_min_max(event_data, event_stats): #use
    event_data['x0'] = (event_data['x0'] - event_stats['x0']['min']) / (event_stats['x0']['max'] - event_stats['x0']['min'])
    event_data['x1'] = (event_data['x1'] - event_stats['x1']['min']) / (event_stats['x1']['max'] - event_stats['x1']['min'])
    event_data['x2'] = (event_data['x2'] - event_stats['x2']['min']) / (event_stats['x2']['max'] - event_stats['x2']['min'])
    return event_data

def inverse_scale_all_features_by_event_min_max(event_data, event_stats):
    event_data['x0'] = event_data['x0'] * (event_stats['x0']['max'] - event_stats['x0']['min']) + event_stats['x0']['min']
    event_data['x1'] = event_data['x1'] * (event_stats['x1']['max'] - event_stats['x1']['min']) + event_stats['x1']['min']
    event_data['x2'] = event_data['x2'] * (event_stats['x2']['max'] - event_stats['x2']['min']) + event_stats['x2']['min']
    return event_data

def normalize_all_features_by_event_z_score(event_data, event_stats):
    event_data['x0'] = (event_data['x0'] - event_stats['x0']['mean']) / event_stats['x0']['std']
    event_data['x1'] = (event_data['x1'] - event_stats['x1']['mean']) / event_stats['x1']['std']
    event_data['x2'] = (event_data['x2'] - event_stats['x2']['mean']) / event_stats['x2']['std']
    return event_data

def inverse_normalize_all_features_by_event_z_score(event_data, event_stats):
    event_data['x0'] = event_data['x0'] * event_stats['x0']['std'] + event_stats['x0']['mean']
    event_data['x1'] = event_data['x1'] * event_stats['x1']['std'] + event_stats['x1']['mean']
    event_data['x2'] = event_data['x2'] * event_stats['x2']['std'] + event_stats['x2']['mean']
    return event_data


def scale_x0_x1_min_max_adjust_x2(event_data, event_stats):
    event_data['x0'] = (event_data['x0'] - event_stats['x0']['min']) / (event_stats['x0']['max'] - event_stats['x0']['min'])
    event_data['x1'] = (event_data['x1'] - event_stats['x1']['min']) / (event_stats['x1']['max'] - event_stats['x1']['min'])
    
    adjusted_x2_min = min(event_stats['x2']['min'], event_stats['x0']['min'], event_stats['x1']['min'])
    adjusted_x2_max = max(event_stats['x2']['max'], event_stats['x0']['max'], event_stats['x1']['max'])
    
    event_data['x2'] = (event_data['x2'] - adjusted_x2_min) / (adjusted_x2_max - adjusted_x2_min)
    return event_data

def inverse_scale_x0_x1_min_max_adjust_x2(event_data, event_stats):
    event_data['x0'] = event_data['x0'] * (event_stats['x0']['max'] - event_stats['x0']['min']) + event_stats['x0']['min']
    event_data['x1'] = event_data['x1'] * (event_stats['x1']['max'] - event_stats['x1']['min']) + event_stats['x1']['min']
    
    adjusted_x2_min = min(event_stats['x2']['min'], event_stats['x0']['min'], event_stats['x1']['min'])
    adjusted_x2_max = max(event_stats['x2']['max'], event_stats['x0']['max'], event_stats['x1']['max'])
    
    event_data['x2'] = event_data['x2'] * (adjusted_x2_max - adjusted_x2_min) + adjusted_x2_min
    return event_data

def normalize_x0_x1_z_score_adjust_x2(event_data, event_stats):
    event_data['x0'] = (event_data['x0'] - event_stats['x0']['mean']) / event_stats['x0']['std']
    event_data['x1'] = (event_data['x1'] - event_stats['x1']['mean']) / event_stats['x1']['std']
    
    adjusted_x2_mean = np.mean([event_stats['x2']['mean'], event_stats['x0']['mean'], event_stats['x1']['mean']])
    adjusted_x2_std = np.mean([event_stats['x2']['std'], event_stats['x0']['std'], event_stats['x1']['std']])
    
    event_data['x2'] = (event_data['x2'] - adjusted_x2_mean) / adjusted_x2_std
    return event_data

def inverse_normalize_x0_x1_z_score_adjust_x2(event_data, event_stats):
    adjusted_x2_mean = np.mean([event_stats['x2']['mean'], event_stats['x0']['mean'], event_stats['x1']['mean']])
    adjusted_x2_std = np.mean([event_stats['x2']['std'], event_stats['x0']['std'], event_stats['x1']['std']])
    
    event_data['x0'] = event_data['x0'] * event_stats['x0']['std'] + event_stats['x0']['mean']
    event_data['x1'] = event_data['x1'] * event_stats['x1']['std'] + event_stats['x1']['mean']
    
    event_data['x2'] = event_data['x2'] * adjusted_x2_std + adjusted_x2_mean
    return event_data
    
def scale_x2_by_combined_x0_x1_range(event_data, event_stats):
    min_x01 = (event_stats['x0']['min'] + event_stats['x1']['min']) / 2
    max_x01 = (event_stats['x0']['max'] + event_stats['x1']['max']) / 2

    min_x2, max_x2 = event_stats['x2']['min'], event_stats['x2']['max']
    scale_factor = (max_x01 - min_x01) / (max_x2 - min_x2)
    
    event_data['x2'] = (event_data['x2'] - min_x2) * scale_factor + min_x01
    return event_data

def inverse_scale_x2_by_combined_x0_x1_range(event_data, event_stats):
    min_x01 = (event_stats['x0']['min'] + event_stats['x1']['min']) / 2
    max_x01 = (event_stats['x0']['max'] + event_stats['x1']['max']) / 2

    min_x2, max_x2 = event_stats['x2']['min'], event_stats['x2']['max']
    scale_factor = (max_x2 - min_x2) / (max_x01 - min_x01)
    
    event_data['x2'] = (event_data['x2'] - min_x01) * scale_factor + min_x2
    return event_data



def load_LC_data(root_name):

    file_path = f'rootfiles/{root_name}.root'
    with uproot.open(file_path) as file:
        tree = file["ticlDumper/clusters;1"]
        branches = tree.arrays(['position_x', 'position_y', 'position_z', 'energy'])
        data_points_number = len(branches['position_x'])
    return branches

def get_event_LC_data(event_id, branches):

    x0, x1, x2 = branches['position_x'][event_id], branches['position_y'][event_id], branches['position_z'][event_id]
    weight = branches['energy'][event_id]
    event_data = pd.DataFrame({'x0': x0, 'x1': x1, 'x2': x2, 'weight': weight, 'indices': list(range(len(x0)))})   
    return event_data

def store_stat_values(root_name, event_id, event_data):

    # Store min, max, mean, and std for each feature (x0, x1, x2)
    Original_stat_values[root_name][event_id] = {
        'x0': {
            'min': event_data['x0'].min(),
            'max': event_data['x0'].max(),
            'mean': event_data['x0'].mean(),
            'std': event_data['x0'].std()
        },
        'x1': {
            'min': event_data['x1'].min(),
            'max': event_data['x1'].max(),
            'mean': event_data['x1'].mean(),
            'std': event_data['x1'].std()
        },
        'x2': {
            'min': event_data['x2'].min(),
            'max': event_data['x2'].max(),
            'mean': event_data['x2'].mean(),
            'std': event_data['x2'].std()
        }
    }








def scale_event_data(event_data, scale_method, event_stats):
    # Dictionary of available scaling methods
    scaling_methods = {
        'x2 by event min max': scale_x2_by_event_min_max,
        'x2 by event z-score': normalize_x2_by_event_z_score,
        'all features by event min max': scale_all_features_by_event_min_max,
        'all features by event z-score': normalize_all_features_by_event_z_score,
        'x0 x1 min max adjust x2': scale_x0_x1_min_max_adjust_x2,
        'x0 x1 z-score adjust x2': normalize_x0_x1_z_score_adjust_x2,
        'x2 by combined x0 x1 range': scale_x2_by_combined_x0_x1_range
    }
    
    if scale_method not in scaling_methods:
        raise ValueError(f"Unknown scaling method: {scale_method}")
    
    # Apply the selected scaling method with the relevant event statistics
    return scaling_methods[scale_method](event_data, event_stats)

def inverse_scale_event_data(event_data, scale_method, event_stats):
    # Dictionary of available inverse scaling methods
    inverse_scaling_methods = {
        'x2 by event min max': inverse_scale_x2_by_event_min_max,
        'x2 by event z-score': inverse_normalize_x2_by_event_z_score,
        'all features by event min max': inverse_scale_all_features_by_event_min_max,
        'all features by event z-score': inverse_normalize_all_features_by_event_z_score,
        'x0 x1 min max adjust x2': inverse_scale_x0_x1_min_max_adjust_x2,
        'x0 x1 z-score adjust x2': inverse_normalize_x0_x1_z_score_adjust_x2,
        'x2 by combined x0 x1 range': inverse_scale_x2_by_combined_x0_x1_range
    }
    
    if scale_method not in inverse_scaling_methods:
        raise ValueError(f"No inverse scaling function defined for method: {scale_method}")
    
    # Apply the selected inverse scaling method with the relevant event statistics
    return inverse_scaling_methods[scale_method](event_data, event_stats)


def apply_save_clustering(root_name, event_id, cluestering_input_df, paramset, output_folder):

    clust = clue.clusterer(paramset[0], paramset[1], paramset[2])
    clust.read_data(cluestering_input_df)
    clust.run_clue()
    #clust.cluster_plotter()
    os.makedirs(output_folder, exist_ok=True)
    output_path = root_name + "/CLUESteringResults_" + str(event_id) + ".csv"
    # print(f"\n\nEvent {event_id} successfully clustered and written in {output_path}")
    clust.to_csv('./CLUEsteringOutput/',output_path)
    clustering_output_path = './CLUEsteringOutput/'+output_path
    return clustering_output_path



def save_grid_search_result_max_based(event_reco, dc, rhoc, dm, total_number_LCs, number_clustered_LCs, non_clustered_points_per_config):

    event_data_tracksters = event_reco['tracksters']
    if event_data_tracksters:
        max_energy_trackster = max(event_data_tracksters, key=lambda t: t['total_energy'])
        max_energy_tackster_size = max_energy_trackster['size']
        print(f"Single Particle => Metric Max Energy trackster Size with params {dc, rhoc, dm} is {max_energy_tackster_size}\n")
    else:
        max_energy_tackster_size = 0  # Default to 0 if no tracksters found

    # Append this parameter combination's size to event_parameters
    event_parameters = {
        "dc": dc,
        "rhoc": rhoc,
        "dm": dm,
        "max_energy_trackster_size": max_energy_tackster_size,
        "points_per_config": total_number_LCs,
        "clustered_points_per_config" : number_clustered_LCs,
        "non_clustered_points_per_config": non_clustered_points_per_config,
        "number_of_clusters_per_config" :len(event_data_tracksters)
    }
    return event_parameters

def sim_to_reco_score(sim_indices, reco_indices, event_data, fractionSim):
    numerator_sum, denominator_sum = 0.0, 0.0   
    shared_energy_sim_to_reco = 0.0
    # Loop through each LC in Sim trackster
    for LC, frac_sim in zip(list(sim_indices), fractionSim): 
        
        # Retrieve LC_energy from Raw_data_before_clustering
        lc_energy = next((event_data['weight'][i] for i, raw_idx in enumerate(event_data['indices']) if raw_idx == LC), 0)

        if LC in reco_indices:
            # case of overlapping LC 
            frac_reco = 1                        
        else:
            frac_reco = 0
        
        point_numerator = min((frac_reco - frac_sim) ** 2, frac_sim ** 2) * lc_energy ** 2
        point_denominator = frac_sim ** 2 * lc_energy ** 2
        shared_energy_sim_to_reco += frac_reco*(frac_sim*lc_energy)
        numerator_sum += point_numerator
        denominator_sum += point_denominator                    
                        
    score = numerator_sum / denominator_sum
    
    # Return calculated values
    return round(score, 8), round(shared_energy_sim_to_reco, 8)

def process_sim_trackster_efficiency(sim_trackster, sim_metrics, efficiency_threshold):
    
    reco_ids = [
        i for i in range(len(sim_metrics["sim to reco fraction"]))
        if sim_metrics["sim to reco fraction"][i] > efficiency_threshold
    ]
    
    if reco_ids:  # Efficient if at least one reco trackster meets the threshold
        return True, sim_metrics
    return False, None

def event_sim_to_reco(event_id, event_data, sim_id, sim_trackster, reco_tracksters):
    sim_indices = sim_trackster['layer_cluster_indices']
    sim_energy = sim_trackster['total_energy']
    sim_size = sim_trackster['size']
    # print(f"------------Processing Sim trackster {sim_id}------------") 
    
    # to store list of scores and shared energies of each reco trackster 
    sim_to_reco_scores_list = []   
    sim_to_reco_shared_energy_list = []
    sim_to_reco_shared_energy_fraction_list = []
    
    # print(f"Sim Trackster ID: {sim_id}")
    # print(f"Sim Total Energy: {sim_energy}")
    # print(f"Sim Trackster Size: {sim_size}")

    # Calculate fractionSim based on vertices multiplicity
    vertices_multiplicity = sim_trackster['vertices_multiplicity']
    fractionSim = [1.0 / vm for vm in vertices_multiplicity]    
                  
    for reco_trackster in reco_tracksters:
        reco_id = reco_trackster['trackster_id']
        reco_indices = reco_trackster['layer_cluster_indices']
        reco_energy = reco_trackster['total_energy']
        reco_size = reco_trackster['size']
        # print(f"   ---Processing Sim trackster {sim_id} with Reco trackster {reco_id}---")

        score, shared_energy_sim_to_reco = sim_to_reco_score(sim_indices, reco_indices, event_data, fractionSim)
        
        sim_to_reco_scores_list.append(score)
        sim_to_reco_shared_energy_list. append(shared_energy_sim_to_reco)
        sim_to_reco_shared_energy_fraction_list.append(shared_energy_sim_to_reco/sim_energy)
        # print(f"   Reco Trackster ID: {reco_id}")
        # print(f"   Reco Total Energy: {sim_energy}")
        # print(f"   Reco Trackster Size: {sim_size}")             
        # print(f"   Score:{score}, shared energy:{shared_energy_sim_to_reco}, fraction:{sim_to_reco_shared_energy_fraction_list[-1]}")

    reco_ids = [i for i in range(len(sim_to_reco_scores_list)) if sim_to_reco_scores_list[i] < merge_threshold]
    sim_metrics = {
        "trackster id":sim_id,
        "sim energy":sim_energy,
        "reco ids": reco_ids,
        "scores":sim_to_reco_scores_list,
        "shared energy":sim_to_reco_shared_energy_list,
        "sim to reco fraction":sim_to_reco_shared_energy_fraction_list
    }
    return sim_metrics


def find_merge(sim_to_merged_reco_idx, reco_trackster_number):
            
    flattened_sim_to_merged_reco_idx = [reco_idx for reco_list in sim_to_merged_reco_idx for reco_idx in reco_list]

    # Count occurrences using Counter or make a hashmap
    occurrences_reco_ids = Counter(flattened_sim_to_merged_reco_idx)
   
    # Find Reco IDs that occur in more than one sim trackster
    reco_ids_with_multiple_occurrences = [key for key, val in occurrences_reco_ids.items() if val > 1]    
    # print(f"Reco IDs that occur in more than one sim trackster:", reco_ids_with_multiple_occurrences)       
    
    merge_rate = len(reco_ids_with_multiple_occurrences)/reco_trackster_number
    return merge_rate


def save_grid_search_result_merge_based(event_id, event_data, event_reco, event_sim, dc, rhoc, dm, merge_threshold, efficiency_threshold):
    if event_reco['tracksters']:
        sim_to_merged_reco_idx = []
        efficient_sim_tracksters_list = []
        num_efficient_sim_tracksters = 0
        sim_track_number = event_sim['number_of_tracksters']
        
        # Loop over each sim trackster
        for sim_trackster in event_sim['tracksters']:            
            sim_id = sim_trackster['trackster_id']
            reco_tracksters = event_reco['tracksters']
            reco_trackster_number = event_reco['number_of_tracksters']

            # print(f"\n\nFor {dc, rhoc, dm}:")
            sim_metrics = event_sim_to_reco(event_id, event_data, sim_id, sim_trackster, reco_tracksters)
            sim_to_merged_reco_idx.append(sim_metrics['reco ids'])
            
            #Efficiency:
            is_efficient, efficient_metrics = process_sim_trackster_efficiency(sim_trackster, sim_metrics, efficiency_threshold)
            if is_efficient:
                efficient_sim_tracksters_list.append(efficient_metrics)
                num_efficient_sim_tracksters += 1
                
        merge_rate = find_merge(sim_to_merged_reco_idx, reco_trackster_number)
        efficiency = num_efficient_sim_tracksters/ sim_track_number
        
    else:
        merge_rate = 1
        efficiency = 0

    # Append this parameter combination's size to event_parameters
    event_parameters = {
        "dc": dc,
        "rhoc": rhoc,
        "dm": dm,
        "merge rate": merge_rate, 
        "efficiency": efficiency
    }
    return event_parameters

def CLUEstering_trackster_properties(root_name, event_id, event_data, grouped_data, dc, rhoc, dm, multi_particle, merge_threshold, efficiency_threshold, event_params_metrics):

    CLUEstering_tracksters_list = []
    total_number_LCs, number_clustered_LCs = 0, 0
    non_clustered_points_per_config = 0

    for cluster_id, group in grouped_data:
        total_number_LCs += len(group)
        
        if cluster_id == -1:
            # print(f"  No. of Outliers is {len(group)}")
            continue 

        total_energy = group['weight'].sum()
        barycenter = (
            (group['x0'] * group['weight']).sum() / total_energy,
            (group['x1'] * group['weight']).sum() / total_energy,
            (group['x2'] * group['weight']).sum() / total_energy
        )
        
        size = len(group)
        # print(f"  No. of Clustered LCs in cluster {cluster_id} is {size}")
        
        layer_indices = group['indices'].tolist()
        
        min_x2_idx, max_x2_idx = group['x2'].idxmin(), group['x2'].idxmax()
        CLUEstering_tracksters_list.append({
            "trackster_id": cluster_id,
            "first_layer_cluster": group.loc[min_x2_idx, ['x0', 'x1', 'x2', 'weight', 'indices']].to_dict(),
            "last_layer_cluster": group.loc[max_x2_idx, ['x0', 'x1', 'x2', 'weight', 'indices']].to_dict(),
            "layer_cluster_indices": layer_indices,
            "total_energy": total_energy,
            "size": size,
            "barycenter_x": barycenter[0],
            "barycenter_y": barycenter[1],
            "barycenter_z": barycenter[2]
        })
        # print(f"  Trackster {cluster_id} has energy {total_energy} and size {size}")
        
        number_clustered_LCs += size
        
        
    # print(f'Total LCs in Event {total_number_LCs}')
    # print(f'No. of clustered LCs {number_clustered_LCs}')

    fraction = number_clustered_LCs / total_number_LCs if total_number_LCs > 0 else 0

    CLUEstering_results[root_name][event_id] = {
        "number_of_tracksters": len(CLUEstering_tracksters_list),
        "tracksters": CLUEstering_tracksters_list,
        "fraction": fraction
    }
    # print(f"No. of tracksters: {CLUEstering_results[root_name][event_id]['number_of_tracksters']}")
  
    event_reco = CLUEstering_results[root_name][event_id]
    
    if multi_particle:
        event_sim = sim_results[root_name][event_id]
        event_param_metric = save_grid_search_result_merge_based(event_id, event_data, event_reco, event_sim, dc, rhoc, dm, merge_threshold, efficiency_threshold)
    else:
        event_param_metric = save_grid_search_result_max_based(event_reco, dc, rhoc, dm, total_number_LCs, number_clustered_LCs, non_clustered_points_per_config)
    
    # print("Event parameters: ", event_param_metric, "\n")
    event_params_metrics.append(event_param_metric)
    # print("Event params: " , event_params_metrics)
    
    return event_params_metrics


def store_stat_values(root_name, event_id, event_data):

    # Ensure that Original_stat_values is initialized for the current root_name
    if root_name not in Original_stat_values:
        Original_stat_values[root_name] = {}

    if event_data.empty:
        # print(f"Event {event_id} of {root_name} has no data. Skipping statistics calculation.")
        return  # Skip storing statistics for empty events

    # Store min, max, mean, and std for each feature (x0, x1, x2)
    Original_stat_values[root_name][event_id] = {
        'x0': {
            'min': event_data['x0'].min(),
            'max': event_data['x0'].max(),
            'mean': event_data['x0'].mean(),
            'std': event_data['x0'].std()
        },
        'x1': {
            'min': event_data['x1'].min(),
            'max': event_data['x1'].max(),
            'mean': event_data['x1'].mean(),
            'std': event_data['x1'].std()
        },
        'x2': {
            'min': event_data['x2'].min(),
            'max': event_data['x2'].max(),
            'mean': event_data['x2'].mean(),
            'std': event_data['x2'].std()
        }
    }
    
# Define the cost function outside of find_optimum_with_cost_function
def cost_function(param, w_merge_rate, w_efficiency):
    merge_rate = param['merge rate']
    efficiency = param['efficiency']
    # Penalize merge rate heavily, reward efficiency lightly
    cost = w_merge_rate * merge_rate - w_efficiency * efficiency
    return cost


def find_optimum_with_cost_function():
    # Dictionary to store the best parameter set for each event
    event_best = {}
    # Dictionary to aggregate cost scores for each parameter set across all events
    overall_scores = {}

    # Weights for the cost function
    w_merge_rate = 0.9  # Focus on minimizing merge rate
    w_efficiency = 0.1  # Less weight on efficiency

    for root, events in all_events_grid_data.items():
        for event_id, param_list in events.items():
            # Filter out parameter sets with efficiency below 0.7
            valid_params = [param for param in param_list if param['efficiency'] >= 0.7]
            
            # If no valid parameters exist, skip this event
            if not valid_params:
                continue
            
            # Find the best parameter set for this event based on the cost function
            best_param = min(valid_params, key=lambda param: cost_function(param, w_merge_rate, w_efficiency))
            event_best[event_id] = best_param
            
            # Aggregate cost scores for each parameter set to find the overall best
            for param in valid_params:
                param_tuple = tuple(param.items())
                if param_tuple not in overall_scores:
                    overall_scores[param_tuple] = 0
                overall_scores[param_tuple] += cost_function(param, w_merge_rate, w_efficiency)
    
    # If there are no valid parameters overall, handle it gracefully
    if not overall_scores:
        print("No valid parameters found across all events.")
        return
    
    # Find the parameter set with the lowest total cost across all events
    overall_best_tuple = min(overall_scores, key=overall_scores.get)
    overall_best = dict(overall_best_tuple)
    
    # Save event_best to a CSV file
    with open('event_best_with_cost.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write header
        writer.writerow(['Event', 'dc', 'rhoc', 'dm', 'merge rate', 'efficiency'])
        
        # Write each event's best parameters
        for event_id, params in event_best.items():
            writer.writerow([event_id, params['dc'], params['rhoc'], params['dm'], params['merge rate'], params['efficiency']])
    
    print("Best parameters per event saved to event_best_with_cost.csv")
    print("\nOverall best parameters:", overall_best)

def read_store_tracksters_properties(root_name, event_id, raw_data_df, vertices_indexes, vertices_energy, barycenters, data_type, vertices_multiplicity=None):

    # Declarations    
    tracksters_list = []
    tracksters_energy = {}
    results = {
        root_name: {
            event_id: {}
        }
    }
    
    print(f"Processing {data_type} Event {event_id} in {root_name} with {len(vertices_indexes)} tracksters...")
    for trackster_id, (trackster_indexes, trackster_energies) in enumerate(zip(vertices_indexes, vertices_energy)):

        total_energy = sum(trackster_energies)
        tracksters_energy[trackster_id] = {
            'total_energy': total_energy,
            'vertices_indexes': trackster_indexes
        }
        filtered_data = raw_data_df[raw_data_df['indices'].isin(trackster_indexes)]

        # Find indices of rows with minimum and maximum x2 values
        min_x2_idx = filtered_data['x2'].idxmin()
        max_x2_idx = filtered_data['x2'].idxmax()

        # Retrieve the rows with min and max x2 values
        first_layer = (filtered_data.loc[min_x2_idx, ['x0', 'x1', 'x2', 'weight', 'indices']]).to_dict()
        last_layer = (filtered_data.loc[max_x2_idx, ['x0', 'x1', 'x2', 'weight', 'indices']]).to_dict()

        trackster_info = {
            'trackster_id': trackster_id,
            'total_energy': total_energy,
            'layer_cluster_indices': trackster_indexes.tolist(),
            'size': len(trackster_indexes),
            'first_layer_cluster': first_layer,
            'last_layer_cluster': last_layer,
            'barycenter_x': barycenters[0][trackster_id],
            'barycenter_y': barycenters[1][trackster_id],
            'barycenter_z': barycenters[2][trackster_id],
            'barycenter_eta': barycenters[3][trackster_id],
            'barycenter_phi': barycenters[4][trackster_id]
        }

        # Include vertices_multiplicity if available
        if vertices_multiplicity is not None:
            trackster_info['vertices_multiplicity'] = vertices_multiplicity[trackster_id].tolist()

        tracksters_list.append(trackster_info)        
        
    # Store results for all tracksters
    results[root_name][event_id] = {
        "number_of_tracksters": len(tracksters_list),
        "tracksters": tracksters_list
    }
    if data_type == "CLUE3D":
        CLUE3D_results[root_name][event_id] = results[root_name][event_id]
    elif data_type == "sim":
        sim_results[root_name][event_id] = results[root_name][event_id]
    else:
        raise ValueError(f"Invalid data_type: {data_type}. Expected string of 'CLUE3D' or 'sim'.")


def load_GT_data(root_name, tree_name):

    file_path = f'rootfiles/{root_name}.root'
    with uproot.open(file_path) as file:
        tree = file[tree_name]
        branches = tree.arrays(['vertices_indexes', 'vertices_energy','barycenter_x','barycenter_y',
                                'barycenter_z','vertices_multiplicity','barycenter_eta','barycenter_phi'])      
    return branches

def get_event_GT_data(event_id, branches):
    return {
        "vertices_indexes": branches["vertices_indexes"][event_id],
        "vertices_energy": branches["vertices_energy"][event_id],
        "barycenter_x": branches["barycenter_x"][event_id],
        "barycenter_y": branches["barycenter_y"][event_id],
        "barycenter_z": branches["barycenter_z"][event_id],
        "vertices_multiplicity": branches["vertices_multiplicity"][event_id],
        "barycenter_eta": branches["barycenter_eta"][event_id],
        "barycenter_phi": branches["barycenter_phi"][event_id],
    }

def particle_Sim():

    for root_name in root_names:
        sim_results[root_name] = {}
        Raw_data_before_clustering[root_name] = {}

        #get branches for both trees
        sim_branches = load_GT_data(root_name, 'ticlDumper/simtrackstersCP;1')
        CLUEstering_branches = load_LC_data(root_name)
                
        for event_id in range(len(CLUEstering_branches)//2):
            # print(f"**********************************************************************")
            # print(f"\nEvent {event_id} of root {root_name}:")
            sim_event_data = get_event_GT_data(event_id, sim_branches)
            
            event_data = get_event_LC_data(event_id, CLUEstering_branches)
            Raw_data_before_clustering[root_name][event_id] = event_data
            
            if event_data.empty:  # Check if empty
                # print(f"No data found in Event {event_id} of {root_name}.")
                continue

            read_store_tracksters_properties(
                event_id=event_id,
                root_name=root_name,
                raw_data_df=event_data,
                vertices_indexes=sim_event_data['vertices_indexes'],
                vertices_energy=sim_event_data['vertices_energy'],
                barycenters=[sim_event_data['barycenter_x'],sim_event_data['barycenter_y'],sim_event_data['barycenter_z'],
                             sim_event_data['barycenter_eta'],sim_event_data['barycenter_phi']],
                vertices_multiplicity=sim_event_data['vertices_multiplicity'] if sim_event_data['vertices_multiplicity'] is not None else None,
                data_type="sim"
            )



def particle_CLUEStering():

    for root_name in root_names:
        CLUEstering_results[root_name] = {}
        Original_stat_values[root_name] = {}  # Ensure this is initialized
        Scaled_data_before_clustering[root_name] = {}
        CLUEstering_max_energy_tracksters[root_name] = {}
        all_events_grid_data[root_name] = {}
        
        CLUEstering_branches = load_LC_data(root_name)
        
        for event_id in range(len(CLUEstering_branches)//2):
            print(f"**********************************************************************")
            print(f"\nEvent {event_id} of root {root_name}:")

            event_data = Raw_data_before_clustering[root_name][event_id]

            if event_data.empty:  # Check if empty
                # print(f"No tracksters found in Event {event_id} of {root_name}.")
                continue

            # Only call store_stat_values if the event data is not empty
            store_stat_values(root_name, event_id, event_data)
            event_stats = Original_stat_values[root_name][event_id]
            
            # Scale data based on the specified scaling method
            event_data = scale_event_data(event_data, scale_method, event_stats)
            Scaled_data_before_clustering[root_name][event_id] = event_data

            # Save the event_data DataFrame to a CSV file after scaling
            output_folder = f'./ScaledEventData/{root_name}'
            os.makedirs(output_folder, exist_ok=True)  # Create the directory if it doesn't exist
            event_data.to_csv(os.path.join(output_folder, f'event_data_{event_id}.csv'), index=False)


            # List to store parameter configurations and sizes for this event
            event_parameters = []
            event_params_metrics = []
            data = Raw_data_before_clustering[root_name][event_id]
            
            for dc, rhoc, dm in product(dc_list, rhoc_list, dm_list):
                
                # Run clustering
                output_folder = f'./CLUEsteringOutput/{root_name}'
                paramset = [dc, rhoc, dm]
                clustering_output_path = apply_save_clustering(root_name, event_id, event_data, paramset, output_folder)
                
                # Process clustering output
                clustering_output_df = pd.read_csv(clustering_output_path)
                
                # Inverse scale the data using the corresponding inverse function
                scaled_back_data = inverse_scale_event_data(clustering_output_df, scale_method, event_stats)
                
                # Merge scaled back data with weights for trackster properties
                merged_df = pd.concat([scaled_back_data, pd.DataFrame({'weight': event_data['weight']})], axis=1)
                merged_df['indices'] = Scaled_data_before_clustering[root_name][event_id]['indices']
    
                # Calculate trackster properties
                grouped = merged_df.groupby('cluster_ids')
                CLUEstering_trackster_properties(root_name, event_id, data, grouped, dc, rhoc, dm, multi_particle, merge_threshold, efficiency_threshold, event_params_metrics)             
            
            all_events_grid_data[root_name][event_id] = event_params_metrics
            # print("all_events_grid_data:\n", all_events_grid_data)



if __name__ == "__main__":
    # Global dictionaries to store results
    Raw_data_before_clustering = {}
    Original_stat_values = {}
    Scaled_data_before_clustering = {}
    CLUEstering_results = {}
    CLUEstering_max_energy_tracksters = {}
    CLUE3D_results = {}
    CLUE3D_max_energy_tracksters = {}
    sim_results = {}
    sim_max_energy_tracksters = {}
    Sim_to_Reco_metrics = {}
    Reco_to_Sim_metrics = {}
    corresponding_tracksters_ids = {}
    all_events_grid_data = {}

    root_names = ['histo_two']
    dc_list = [0.2, 0.5, 0.7]
    rhoc_list = [4, 4.5, 5]
    dm_list = [0.5, 1, 2]
    merge_threshold = 0.6
    efficiency_threshold = 0.5
    multi_particle = True
    scale_method = 'all features by event z-score'  # Choose the desired scaling method
    
    # Step 2: Load the simulation data
    particle_Sim()
    
    # Step 1: Process the data with CLUEstering
    print(f"\nStarting CLUEstering processing...\n")
    particle_CLUEStering()

    # Step 3: Find the Optimum set of parameters
    print(f"\nFinding the Optimum set of parameters with this scaling")
    # find_optimum()
    find_optimum_with_cost_function()

    print(f"Pipeline completed.")


Processing sim Event 1 in histo_two with 2 tracksters...
Processing sim Event 2 in histo_two with 2 tracksters...
Processing sim Event 3 in histo_two with 2 tracksters...
Processing sim Event 4 in histo_two with 2 tracksters...
Processing sim Event 5 in histo_two with 2 tracksters...
Processing sim Event 6 in histo_two with 2 tracksters...
Processing sim Event 7 in histo_two with 2 tracksters...
Processing sim Event 8 in histo_two with 2 tracksters...
Processing sim Event 9 in histo_two with 2 tracksters...
Processing sim Event 10 in histo_two with 2 tracksters...
Processing sim Event 11 in histo_two with 2 tracksters...
Processing sim Event 12 in histo_two with 2 tracksters...
Processing sim Event 13 in histo_two with 2 tracksters...
Processing sim Event 14 in histo_two with 2 tracksters...
Processing sim Event 15 in histo_two with 2 tracksters...
Processing sim Event 16 in histo_two with 2 tracksters...
Processing sim Event 17 in histo_two with 2 tracksters...
Processing sim Event 18